COVID - DATA EXTRACTION FROM AN ONTOLOGY

In [ ]:
pip install rdflib

In [ ]:
#INTRODUCTION
"""
The objective of this script is to perform the extraction of tables in the OMOP v5.3 format from an ontology 
that contains medical data. 
"""
#LIBRARIES
import tkinter as tk 
from tkinter import filedialog
import rdflib
import pandas as pd

In [ ]:
#IMPORT ALL OWL PATHS

root = tk.Tk() 
root.withdraw() 

"""
filedialog open a box which allows the user to open one or more files
filetypes filter the type of files the user can open
multiple = True allows the user to select multiple files
"""

selected_files = filedialog.askopenfiles(filetypes=[("OWL files", "*.owl")], multiple=True) 

#list with all the file names which contains selected_files
file_paths = [file.name for file in selected_files]

In [ ]:
#GENERATE THE GRAPH USING ALL OWL PATHS

total_graph = rdflib.Graph() #Generates an empty Graph

#Fill the graph with the owls data, the graph will have the format xml
for path in file_paths:
    total_graph.parse(path, format='xml')

In [ ]:
#EXTRACT THE NAMESPACES OF THE ONTOLOGY

"""
In this part of the code we extract the namespaces of the ontology to build the query. This can be easily done 
thanks to the graph.namespaces() of rdflib
"""

def obtener_prefijos_namespaces(graph):
    prefijos_namespaces = []
    for prefix, namespace in graph.namespaces():
        prefijo_namespace = f"prefix {prefix}: <{namespace}>"
        prefijos_namespaces.append(prefijo_namespace)
    
    return "\n".join(prefijos_namespaces)

Q_prefix = obtener_prefijos_namespaces(total_graph)
#print(Q_prefix)

In [ ]:
#SELECT CREATOR
"""
This function creates the SELECT part of an SPARQL query using a list as input.
"""


def convert_to_sparql_select(variables):
    select_clause = "SELECT "
    for variable in variables:
        select_clause += "?" + variable + " "
    return select_clause.rstrip()

In [ ]:
#LISTS WITH ALL THE VARIABLES OF THE DIFFERENT OMOP CDM V5.3 TABLES

person = ["person_id", "gender_concept_id", "year_of_birth", "month_of_birth", "day_of_birth",
"birth_datetime", "race_concept_id", "ethnicity_concept_id", "location_id", "provider_id", "care_site_id",
"person_source_value", "gender_source_value", "gender_source_concept_id", "race_source_value", 
"race_source_concept_id", "ethnicity_source_value", "ethnicity_source_concept_id"]

condition_occurrence = ["condition_occurrence_id", "person_id", "condition_concept_id",
"condition_start_date", "condition_start_datetime", "condition_end_date", "condition_end_datetime",
"condition_type_concept_id", "condition_status_concept_id", "stop_reason", "provider_id",
"visit_occurrence_id", "visit_detail_id", "condition_source_value", "condition_source_concept_id",
"condition_status_source_value"]

drug_exposure = ["drug_exposure_id", "person_id", "drug_concept_id", "drug_exposure_start_date",
"drug_exposure_start_datetime", "drug_exposure_end_date", "drug_exposure_end_datetime", "verbatim_end_date",
"drug_type_concept_id", "stop_reason", "refills", "quantity", "days_supply", "sig", "route_concept_id",
"lot_number", "provider_id", "visit_occurrence_id", "visit_detail_id", "drug_source_value", 
"drug_source_concept_id", "route_source_value", "dose_unit_source_value"]

measurement =["measurement_id", "person_id", "measurement_concept_id", "measurement_date",
"measurement_datetime", "measurement_time", "measurement_type_concept_id", "operator_concept_id",
"value_as_number", "value_as_concept_id", "unit_concept_id", "range_low", "range_high", "provider_id",
"visit_occurrence_id", "visit_detail_id", "measurement_source_value", "measurement_source_concept_id",
"unit_source_value", "value_source_value"]


procedure_occurrence = ["procedure_occurrence_id", "person_id", "procedure_concept_id",
"procedure_date", "procedure_datetime", "procedure_type_concept_id", "modifier_concept_id",
"quantity", "provider_id", "visit_occurrence_id", "visit_detail_id", "procedure_source_value",
"procedure_source_concept_id", "modifier_source_value"]

observation = ["observation_id", "person_id", "observation_concept_id", "observation_date",
"observation_datetime", "observation_type_concept_id", "value_as_number", "value_as_string", "value_as_concept_id",
"qualifier_concept_id", "unit_concept_id", "provider_id", "visit_occurrence_id", "visit_detail_id",
"observation_source_value", "observation_source_concept_id", "unit_source_value","qualifier_source_value"]


visit_detail = ["visit_detail_id", "person_id", "visit_detail_concept_id", "visit_detail_start_date",
"visit_detail_start_datetime", "visit_detail_end_date", "visit_detail_end_datetime", 
"visit_detail_type_concept_id", "provider_id", "care_site_id", "visit_detail_source_value", 
"visit_detail_source_concept_id", "admitting_source_value", "admitting_source_concept_id", 
"discharge_to_source_value", "discharge_to_concept_id", "preceding_visit_detail_id", "visit_detail_parent_id",
"visit_occurrence_id"]


visit_occurrence = ["visit_occurrence_id", "person_id", "visit_concept_id", "visit_start_date",
"visit_start_datetime", "visit_end_date", "visit_end_datetime", "visit_type_concept_id", "provider_id",
"care_site_id", "visit_source_value", "visit_source_concept_id", "admitting_source_concept_id",
"admitting_source_value", "discharge_to_concept_id", "discharge_to_source_value", "preceding_visit_occurrence_id"]



In [ ]:
#QUERYS TO EXTRACT THE VALUES

Q_PERSON = Q_prefix + convert_to_sparql_select(person) + """
WHERE
{
#Query1
}
"""

Q_CONDITION_OCCURRENCE = Q_prefix + convert_to_sparql_select(condition_occurrence) + """
WHERE{
#Query2
}
"""

Q_MEASUREMENT = Q_prefix + convert_to_sparql_select(measurement) + """
WHERE{
#Query3
}
"""

Q_PROCEDURE_OCCURRENCE = Q_prefix + convert_to_sparql_select(procedure_occurrence) + """
WHERE{
#Query4
}
"""

Q_OBSERVATION = Q_prefix + convert_to_sparql_select(observation) + """
WHERE{
#Query5
}
"""

Q_VISIT_DETAIL = Q_prefix + convert_to_sparql_select(visit_detail) + """
WHERE{
#Query6
}
"""

Q_VISIT_OCCURRENCE = Q_prefix + convert_to_sparql_select(visit_occurrence) + """
WHERE{
#Query7
}
"""

In [ ]:
files_path = "/home/guillem/Descargas/TABLAS_CONSULTAS/"

def query_to_csv(g, query, cols, csv_name, path):
    result = g.query(query)
    df = pd.DataFrame(result, columns=cols)
    df.to_csv(path + csv_name + ".csv",index=False)

#PERSON
query_to_csv(total_graph, Q_PERSON, person, "PERSON", files_path)

#CONDITION_OCCURRENCE
query_to_csv(total_graph, Q_CONDITION_OCCURRENCE_prob_sal, 
condition_occurrence, "CONDITION_OCCURRENCE", files_path)

#DRUG_EXPOSURE
query_to_csv(total_graph, Q_DRUG_EXPOSURE_acum, drug_exposure, "DRUG_EXPOSURE", files_path)

#MEASUREMENT
query_to_csv(total_graph, Q_MEASUREMENT_lab, measurement, "MEASUREMENT", files_path)

#PROCEDURE_OCCURRENCE
query_to_csv(total_graph, Q_PROCEDURE_OCCURRENCE, procedure_occurrence, "PROCEDURE_OCCURRENCE", files_path)

#OBSERVATION
query_to_csv(total_graph, Q_OBSERVATION, observation, "OBSERVATION", files_path)

#VISIT_DETAIL
query_to_csv(total_graph, Q_VISIT_DETAIL, visit_detail, "VISIT_DETAIL", files_path)

#VISIT_OCCURRENCE
query_to_csv(total_graph, Q_VISIT_OCCURRENCE, visit_occurrence, "VISIT_OCCURRENCE", files_path)